In [2]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import pyodbc

In [3]:
engine = create_engine('sqlite:///chinook.db')
engine

Engine(sqlite:///chinook.db)

In [4]:
def create_date_table(start='2009-01-01', end='2030-12-31'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    df['año'] = df.date.dt.year
    df['mes'] = df.date.dt.month
    df['dia'] = df.date.dt.day
    df['nombre_dia'] = df.date.dt.day_name()
    df['dia_semana'] = df.date.dt.dayofweek
    df['semana'] = df.date.dt.weekofyear
    df['trimestre'] = df.date.dt.quarter
    
    df = df[['date', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']] 
    
    return df

In [5]:
frameTime = create_date_table()

C:\Users\zambr\AppData\Local\Temp/ipykernel_22320/4140598203.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['semana'] = df.date.dt.weekofyear


In [6]:
frameTime.rename(columns={'date':'fecha'},inplace=True)

In [7]:
frameTime

,fecha,año,mes,dia,nombre_dia,dia_semana,semana,trimestre
0,2009-01-01,2009,1,1,Thursday,3,1,1
1,2009-01-02,2009,1,2,Friday,4,1,1
2,2009-01-03,2009,1,3,Saturday,5,1,1
3,2009-01-04,2009,1,4,Sunday,6,1,1
4,2009-01-05,2009,1,5,Monday,0,2,1
...,...,...,...,...,...,...,...,...
8030,2030-12-27,2030,12,27,Friday,4,52,4
8031,2030-12-28,2030,12,28,Saturday,5,52,4
8032,2030-12-29,2030,12,29,Sunday,6,52,4
8033,2030-12-30,2030,12,30,Monday,0,1,4


In [6]:
frameTime.to_sql("Dim_Time", engine.connect(), if_exists='append', index=False)

fin = print("Carga Terminada!!!")

Carga Terminada!!!


In [7]:
frameAlbums = pd.read_sql_query("""select title from albums;""", 
    con=engine.connect())

In [8]:
frameArtists = pd.read_sql_query("""select name from artists;""", 
    con=engine.connect())

In [9]:
frameCustomers = pd.read_sql_query("""select FirstName, LastName,  Phone, Fax, Email from customers;""", 
    con=engine.connect())

In [10]:
frameEmployees = pd.read_sql_query("""select LastName, FirstName, Title, BirthDate, HireDate, Address, City, State,
Country, PostalCode, Phone, Fax, Email from employees;""", 
    con=engine.connect())

In [11]:
frameInItems = pd.read_sql_query("""select UnitPrice, Quantity from invoice_items;""", 
    con=engine.connect())

In [12]:
frameLocation = pd.read_sql_query("""select BillingAddress as Address, BillingCity as City, BillingState as State, BillingCountry as Country,
BillingPostalCode as PostalCode from invoices;""", 
    con=engine.connect())

In [13]:
frameTiempo = pd.read_sql_query("""select fecha, año, mes, dia,
nombre_dia, dia_semana, semana, trimestre from Dim_Time;""", 
    con=engine.connect())

In [14]:
frameTracks = pd.read_sql_query("""select t.name, mt.Name as Mediatype, g.Name as Genre, t.Composer, t.Milliseconds,t.Bytes,t.UnitPrice 
from tracks t inner join media_types mt
on t.MediaTypeId  = mt.MediaTypeId inner join genres g
on t.GenreId = g.GenreId;""", 
    con=engine.connect())

In [15]:
    frameTablaHechos = pd.read_sql_query("""SELECT invoice_items.InvoiceLineId as InvoicesId, customers.CustomerId,
    employees.EmployeeId,dim_time.Timeid,
    invoices.InvoiceId as LocationId, tracks.TrackId, artists.ArtistId, albums.AlbumId, invoices.Total
    FROM invoices INNER JOIN customers
    ON invoices.CustomerId= customers.CustomerId
    INNER JOIN employees
    ON customers.SupportRepId = employees.EmployeeId
    INNER JOIN invoice_items
    ON invoice_items.InvoiceId= invoices.InvoiceId
    INNER JOIN tracks
    ON tracks.TrackId=  invoice_items.TrackId
    INNER JOIN albums
    ON tracks.AlbumId= albums.AlbumId
    INNER JOIN artists 
    ON albums.ArtistId= artists.ArtistId 
    INNER JOIN dim_time
    ON invoices.InvoiceDate = substr(dim_time.fecha,0,20);""", 
    con=engine.connect())

In [16]:
frameAlbums.fillna(method="bfill", inplace = True) 
frameArtists.fillna(method="bfill", inplace = True) 
frameCustomers.fillna(method="bfill", inplace = True) 
frameEmployees.fillna(method="bfill", inplace = True) 
frameInItems.fillna(method="bfill", inplace = True) 
frameLocation.fillna(method="bfill", inplace = True) 
frameTiempo.fillna(method="bfill", inplace = True) 
frameTracks.fillna(method="bfill", inplace = True) 
frameTablaHechos.fillna(method="bfill", inplace = True) 

In [17]:
frameTablaHechos

,InvoicesId,CustomerId,EmployeeId,TimeId,LocationId,TrackId,ArtistId,AlbumId,Total
0,531,1,3,435,98,3247,158,253,3.98
1,532,1,3,435,98,3248,158,253,3.98
2,649,1,3,529,121,447,52,37,3.96
3,650,1,3,529,121,449,52,37,3.96
4,651,1,3,529,121,451,52,37,3.96
...,...,...,...,...,...,...,...,...,...
2235,1415,57,5,1150,262,1602,22,129,5.94
2236,1416,57,5,1150,262,1606,22,130,5.94
2237,1417,57,5,1150,262,1610,22,131,5.94
2238,1418,57,5,1150,262,1614,22,131,5.94


In [18]:
engine = sqlalchemy.create_engine('mssql+pyodbc://localhost/DW_Sales_Music3?driver=SQL+Server+Native+Client+11.0')
try:     
    frameAlbums.to_sql(name='Dim_Album', con=engine, if_exists='append',index=False)
    frameArtists.to_sql('Dim_Artist', con=engine, if_exists='append',index=False)
    frameCustomers.to_sql('Dim_Customer', con=engine, if_exists='append',index=False)
    frameEmployees.to_sql('Dim_Employee', con=engine, if_exists='append',index=False)
    frameInItems.to_sql('Dim_Invoice_Item', con=engine, if_exists='append',index=False)
    frameLocation.to_sql('Dim_Location', con=engine, if_exists='append',index=False)
    frameTiempo.to_sql('Dim_Time', con=engine, if_exists='append',index=False)
    frameTracks.to_sql('Dim_Track', con=engine, if_exists='append',index=False)
    frameTablaHechos.to_sql('Fact_Sales', con=engine, if_exists='append',index=False)
except Exception as ex:
                 print("Ocurrió un error: {}".format(ex))
   